In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from tqdm import tqdm

In [13]:
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(), transforms.Normalize((0.5), (0.5))])
training_set = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
testing_set = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)

In [14]:
training_loader = torch.utils.data.DataLoader(training_set, batch_size=20, shuffle=True, num_workers=2,  pin_memory=True)
testing_loader = torch.utils.data.DataLoader(testing_set, batch_size=20, shuffle=True, num_workers=2, pin_memory=True)

In [30]:
class Model(nn.Module):

  def __init__(self):
    super(Model, self).__init__()
    self.layers = nn.Sequential(

        nn.Conv2d(1, 64, kernel_size=3, padding=1),
        nn.ReLU(),

        nn.MaxPool2d(2, 2),

        nn.Conv2d(64, 128, kernel_size=3, padding=1),
        nn.ReLU(),

        nn.MaxPool2d(2, 2),

        nn.Conv2d(128, 256, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size=3, padding=1),
        nn.ReLU(),

        nn.MaxPool2d(2, 2),

        nn.Conv2d(256, 512, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, padding=1),
        nn.ReLU(),

        nn.MaxPool2d(2, 2),

        nn.Conv2d(512, 512, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, padding=1),
        nn.ReLU(),

        nn.MaxPool2d(2, 2),


        # Classifier
        nn.Flatten(),

        nn.Linear(512*7*7, 4096),
        nn.Dropout(0.2),
        nn.ReLU(),

        nn.Linear(4096, 4096),
        nn.Dropout(0.2),
        nn.ReLU(),

        nn.Linear(4096, 10)

    )


    self.loss = nn.CrossEntropyLoss()
    self.optimizer = torch.optim.Adam(self.parameters(), lr=0.00001, betas=(0.9, 0.999))
    self.scheduler = None

  def forward(self, x):
    return self.layers(x)

  def train_model(self, training_loader, epoch):
    self.scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer, max_lr=0.00002, steps_per_epoch=len(training_loader), epochs=epoch)
    self.train()
    self.to('cuda')
    for i in range(epoch):
      running_loss=0
      for images, labels in tqdm(training_loader):
        images = images.to('cuda')
        labels = labels.to('cuda')
        self.optimizer.zero_grad()
        pred = self.forward(images)
        loss = self.loss(pred, labels)
        loss.backward()
        self.optimizer.step()
        self.scheduler.step()
        running_loss += loss.item()


      print(running_loss/len(training_loader))


  def test_model(self, testing_loader):
      self.eval()
      self.to('cuda')
      true = 0
      total=0
      with torch.no_grad():
        for images, labels in testing_loader:
          images = images.to('cuda')
          labels = labels.to('cuda')
          pred = self.forward(images)
          pred_actual = torch.argmax(pred, dim=-1)
          true += (pred_actual == labels).sum().item()
          total += labels.size(0)

        accuracy = true / total
        print(f"Accuracy: {accuracy * 100:.2f}%")



In [ ]:
print(f"CUDA Available: {torch.cuda.is_available()}")
model = Model().to('cuda')
model.train_model(training_loader, 20)
model.test_model(testing_loader)


CUDA Available: True


100%|████████████████████████████████████████████████████████████████████████████| 3000/3000 [1:31:49<00:00,  1.84s/it]


1.041941740329067


 27%|████████████████████▋                                                        | 804/3000 [25:05<1:04:49,  1.77s/it]